# ColBERTv2: Indexing & Search Notebook

If you're working in Google Colab, we recommend selecting "GPU" as your hardware accelerator in the runtime settings.

First, we'll import the relevant classes. Note that `Indexer` and `Searcher` are the key actors here. Next, we'll download the necessary dependencies.

In [1]:
!git -C ColBERT/ pull || git clone https://github.com/stanford-futuredata/ColBERT.git
import sys; sys.path.insert(0, 'ColBERT/')


fatal: cannot change to 'ColBERT/': No such file or directory
Cloning into 'ColBERT'...
remote: Enumerating objects: 2417, done.
remote: Counting objects: 100% (924/924), done.
remote: Compressing objects: 100% (234/234), done.
remote: Total 2417 (delta 745), reused 695 (delta 690), pack-reused 1493
Receiving objects: 100% (2417/2417), 1.98 MiB | 17.33 MiB/s, done.
Resolving deltas: 100% (1502/1502), done.


In [2]:
try: # When on google Colab, let's install all dependencies with pip.
    import google.colab
    !pip install -U pip
    !pip install -e ColBERT/['faiss-gpu','torch']
except Exception:
  import sys; sys.path.insert(0, 'ColBERT/')
  try:
    from colbert import Indexer, Searcher
  except Exception:
    print("If you're running outside Colab, please make sure you install ColBERT in conda following the instructions in our README. You can also install (as above) with pip but it may install slower or less stable faiss or torch dependencies. Conda is recommended.")
    assert False

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 14.0 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
Obtaining file:///content/ColBERT
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 100.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.3/285.3 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2

In [3]:
import colbert

In [4]:
from colbert import Indexer, Searcher
from colbert.infra import Run, RunConfig, ColBERTConfig
from colbert.data import Queries, Collection

We will use the dev set of the **LoTTE benchmark** we recently introduced in the ColBERTv2 paper. We'll download it from HuggingFace datasets. The dev and test sets contain several domain-specific corpora, and we'll use the smallest dev set corpus, namely lifestyle:dev.

For the purposes of a quick demo, we will only run the `Indexer` on the first 10,000 passages. As we do this, let's also remove the queries whose relevant passages are all outside this small set of passages.

In [5]:
!git clone https://github.com/NickVoulg02/Information-Retrieval

Cloning into 'Information-Retrieval'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 40 (delta 10), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (40/40), 116.17 KiB | 5.28 MiB/s, done.
Resolving deltas: 100% (10/10), done.


In [44]:
from datasets import Dataset
import pandas as pd
dataset = 'test'
df1 = pd.read_csv("Information-Retrieval/test/doc_col.tsv", delimiter = '\t', index_col=0)
df2 =  pd.read_csv("Information-Retrieval/test/queries_20.tsv", delimiter = '\t', index_col=0)
collection = Dataset.from_pandas(df1)
query = Dataset.from_pandas(df2)
print(query['query'][5])

WHAT IS THE EFFECT OF WATER OR OTHER THERAPEUTIC AGENTS ON THE PHYSICAL PROPERTIES VISCOSITY ELASTICITY OF SPUTUM OR BRONCHIAL SECRETIONS FROM CF PATIENTS


This loaded 417 queries and 269k passages. Let's inspect one query and one passage to verify we have done so correctly.

In [ ]:
# print(queries[24])
# print()
# print(collection[19929])
# print()

are blossom end rot tomatoes edible?

I think the spraying thing is not after, it's during. The cold will freeze the mist, keeping the air around the trees at (but not below) freezing. See http://www.ehow.com/how_5805520_use-freeze-damage-fruit-trees.html for example which recommends a sprinkler. The "releases heat" thing is kind of an oversimplification, but basically as long as you have any liquid water around, it will keep things at zero. The sap of your tree is not pure water, and therefore freezes somewhat below zero. By having the water freeze instead you stay away from the temps that would damage your plants. That said, http://www.ehow.com/how-does_5245655_spraying-frost-protect-fruit-freezing_.html is total gibberish since evaporation doesn't generate heat, quite the opposite. There is a better explanation at http://www.gardenguides.com/135830-spray-water-plants-during-frost.html This is a picture from a blog entry that gives you details from the citrus farmer's point of view.


## Indexing

For an efficient search, we can pre-compute the ColBERT representation of each passage and index them.

Below, the `Indexer` take a model checkpoint and writes a (compressed) index to disk. We then prepare a `Searcher` for retrieval from this index.

In [45]:
nbits = 2   # encode each dimension with 2 bits
doc_maxlen = 300 # truncate passages at 300 tokens

index_name = f'{dataset}.{nbits}bits'

To save space and time, we will only run the `Indexer` on the first 10,000 passages. To do so, we will filter out queries that do not contain passages with ids less than 10,000.

In [ ]:
# answer_pids = [x['answers']['answer_pids'] for x in queries_dataset['search_' + datasplit]]
# filtered_queries = [q for q, apids in zip(queries, answer_pids) if any(x < max_id for x in apids)]

# f'Filtered down to {len(filtered_queries)} queries'

'Filtered down to 20 queries'

Now run the `Indexer` on the collection subset. Assuming the use of only one GPU, this cell should take about six minutes to finish running.

In [46]:
checkpoint = 'colbert-ir/colbertv2.0'

with Run().context(RunConfig(nranks=1, experiment='notebook')):  # nranks specifies the number of GPUs to use
    config = ColBERTConfig(doc_maxlen=doc_maxlen, nbits=nbits, kmeans_niters=4) # kmeans_niters specifies the number of iterations of k-means clustering; 4 is a good and fast default.
                                                                                # Consider larger numbers for small datasets.

    indexer = Indexer(checkpoint=checkpoint, config=config)
    indexer.index(name=index_name, collection=collection["doc"], overwrite=True)



[Dec 10, 22:39:37] #> Note: Output directory /content/experiments/notebook/indexes/test.2bits already exists


[Dec 10, 22:39:37] #> Will delete 10 files already at /content/experiments/notebook/indexes/test.2bits in 20 seconds...
#> Starting...
#> Joined...


In [47]:
indexer.get_index() # You can get the absolute path of the index, if needed.

'/content/experiments/notebook/indexes/test.2bits'

## Search

Having built the index and prepared our `searcher`, we can search for individual query strings.

We can use the `queries` set we loaded earlier — or you can supply your own questions. Feel free to get creative! But keep in mind this set of ~300k lifestyle passages can only answer a small, focused set of questions!

In [48]:
# To create the searcher using its relative name (i.e., not a full path), set
# experiment=value_used_for_indexing in the RunConfig.
with Run().context(RunConfig(experiment='notebook')):
    searcher = Searcher(index=index_name, collection=collection["doc"])


# If you want to customize the search latency--quality tradeoff, you can also supply a
# config=ColBERTConfig(ncells=.., centroid_score_threshold=.., ndocs=..) argument.
# The default settings with k <= 10 (1, 0.5, 256) gives the fastest search,
# but you can gain more extensive search by setting larger values of k or
# manually specifying more conservative ColBERTConfig settings (e.g. (4, 0.4, 4096)).

[Dec 10, 22:45:32] #> Loading codec...
[Dec 10, 22:45:32] #> Loading IVF...
[Dec 10, 22:45:32] #> Loading doclens...


100%|██████████| 1/1 [00:00<00:00, 4957.81it/s]

[Dec 10, 22:45:32] #> Loading codes and residuals...



100%|██████████| 1/1 [00:00<00:00, 714.65it/s]


In [49]:
question = query["query"][13] # try with an in-range query or supply your own
print(f"#> {question}")

# Find the top-3 passages for this query
results = searcher.search(question, k=3)

# Print out the top-k retrieved passages
for passage_id, passage_rank, passage_score in zip(*results):
    print(f"\t [{passage_rank}] \t\t {passage_score:.1f} \t\t {searcher.collection[passage_id]}")

#> WHAT NON-INVASIVE TESTS CAN BE PERFORMED FOR THE EVALUATION OF EXOCRINE PANCREATIC FUNCTION IN PATIENTS WITH CF

#> QueryTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
#> Input: . WHAT NON-INVASIVE TESTS CAN BE PERFORMED FOR THE EVALUATION OF EXOCRINE PANCREATIC FUNCTION IN PATIENTS WITH CF, 		 True, 		 None
#> Output IDs: torch.Size([32]), tensor([  101,     1,  2054,  2512,  1011, 17503,  5852,  2064,  2022,  2864,
         2005,  1996,  9312,  1997,  4654, 10085, 11467,  6090, 16748, 12070,
         3853,  1999,  5022,  2007, 12935,   102,   103,   103,   103,   103,
          103,   103])
#> Output Mask: torch.Size([32]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 0, 0, 0, 0, 0, 0])

	 [1] 		 14.6 		 A CLINICAL STUDY OF THE DIAGNOSIS OF CYSTIC FIBROSIS BY INSTRUMENTAL NEUTRON ACTIVATION ANALYSIS OF SODIUM IN NAIL CLIPPINGS THIS ARTICLE REPORTS ON THE POSSIBILITY OF USING INSTRUMENTAL NEUTRON ACTIVATION ANALYSI